# Viterbi version of target sequence labelling
# Intro
The viterbi algorithm for labelling sequences uses the Hidden-Markov-Model and computes the
most probable label sequence as the combination of transition and emission probabilities for
the given word sequence. It is most often used in POS-tagging but nicely fits our task of
labelling emotion target sequences. As it uses not only the observed tokens but also the last
(number of) tag, it takes the probability of tags following each other into account. As our tags
are rigid in their sequence (Inside only follows Beginning, Beginning only follows Outside), using
transition probabilities will improve the output labels by hopefully avoiding illegal tag transitions.
# Method
The used viterbi algorithm is implemented from scratch and works on the non-preprocessed tokens.
To use it, the data has to be prepared first.

In [1]:
import pandas

## Data preparation
As seen later, the algorithm is trained by calculating emission and transition probabilities,
which are taken from the tokenized training data. First, the file is loaded and all unnecessary
parts are dropped.

In [2]:
with open('emotion-rl-corpora.jsonl', encoding='utf-8') as f:
    data_source = pandas.read_json(f, lines=True)
data_source = data_source.loc[
    (data_source['dataset']=='reman')|(data_source['dataset']=='electoral_tweets')|(data_source['dataset']=='gne')
]
noanno = data_source[data_source['annotations'] == {}].index
data_source = data_source.drop(noanno)
data_source = data_source.drop(columns=[
    'meta', 'steps', 'tags', 'split', 'annotation-offsets', 'extra'
])

data_source = data_source.join(data_source['annotations'].apply(pandas.Series))
noanno = [i for i,row in data_source.iterrows() if type(row['target']) == float]
data_source = data_source.drop(noanno)
data_source = data_source.drop(columns=[
    'annotations', 'cause', 'cue-joy', 'cue-sadness', 'cue-anger', 'cue-other', 'cue-disgust',
    'cue-trust', 'cue-anticipation', 'cue-surprise', 'cue-fear', 'cue', 'experiencer'
])

After removing every annotation but target and all additional information other than the corpus and text id,
the raw text itself and token and label lists, the algorithm can be trained.
## Training
### HMM
The viterbi algorithm is based on a Hidden-Markov model, as the model computes the probability of a token-tag sequence
and viterbi is a way to return the tag sequence with the highest probability. Why viterbi is necessary will be explained
after implementing a HMM.

In [3]:
class HiddenMarkovModel:
    transitions = dict()
    emissions = dict()
    prior = dict()
    observations = {'transitions': transitions, 'emissions': emissions}

    def __init__(self):
        self.transitions = {'O': {'O': 0, 'B': 0, 'I': 0}, 'B': {'O': 0, 'B': 0, 'I': 0}, 'I': {'O': 0, 'B': 0, 'I': 0}}
        self.prior = {'O': 0, 'B': 0, 'I': 0}

    def train_probabilities(self, train_set: pandas.DataFrame):
        trans = {'O': {'O': 0, 'B': 0, 'I': 0}, 'B': {'O': 0, 'B': 0, 'I': 0}, 'I': {'O': 0, 'B': 0, 'I': 0}}
        emi = dict()
        prior = {'O': 0, 'B': 0, 'I': 0}
        for index, row in train_set.iterrows():
            # The current sequence consists of (word, tag) pairs
            seq = list(zip(row['tokens'], row['target']))

            for i in range(len(seq)):
                tok = seq[i][0]
                tag = seq[i][1]
                # First tag: Prior count
                if i == 0:
                    prior[tag] += 1
                # Every other tag: Transition count from the previous tag
                else:
                    trans[tag][seq[i - 1][1]] += 1
                # Emission count from token-tag-pairs
                if tok in emi:
                    emi[tok][tag] += 1
                else:
                    emi[tok] = {'O': 0, 'B': 0, 'I': 0}
                    emi[tok][tag] += 1
        # Converting the absolute emission/transition/prior frequencies into probabilities
        for tag in {'O', 'B', 'I'}:
            freq = sum(trans[tag].values())
            for prev_tag in trans[tag]:
                trans[tag][prev_tag] /= freq
            prior[tag] /= sum(prior.values())

        for tok in emi:
            freq = sum(emi[tok].values())
            for tag in {'O', 'B', 'I'}:
                emi[tok][tag] /= freq

        emi['$OOV'] = {tag: sum(trans[tag].values())/sum([sum(trans[tag2].values())
                                                              for tag2 in {'O', 'B', 'I'}]) for tag in {'O', 'B', 'I'}}
        self.transitions = trans
        self.emissions = emi
        self.prior = prior

### Viterbi
The trained HMM includes the emission probabilities as token-tag-pair-frequencies relative to token-frequency and
transition probabilities as tag bigram frequencies relative to the frequency of the second tag. As every token in the
sequence has three tag possibilities which are also influenced by the previous tag, there are 3<sup>n</sup> tag sequences
for a sequence with n tokens, all with differing probabilities. Viterbi keeps track of the maximum probability of a current
token and the maximum probability of different histories to compute the best sequence from all HMM-sequences.

In [4]:
def viterbi(sequence: list[str], model: HiddenMarkovModel):
    viterbi_table = [[0 for _ in range(3)] for _ in range(len(sequence))]
    max_labels = list()
    # Go through the sequence token by token
    for i_tok, token in enumerate(sequence):
        # For each possible Tag, multiply
        for i_tag, tag in enumerate(['O', 'B', 'I']):
            # the emission probability of the tag and current token
            if token in model.emissions:
                viterbi_table[i_tok][i_tag] = model.emissions[token][tag]
            else:
                viterbi_table[i_tok][i_tag] = model.emissions['$OOV'][tag]
            # with the prior probability of the tag for the first token
            if i_tok == 0:
                viterbi_table[i_tok][i_tag] *= model.prior[tag]
            # or with the transition probability for best tag of the last token for all other tokens
            else:
                viterbi_table[i_tok][i_tag] *= max_v * model.transitions[tag][max_t]
        max_v = max(viterbi_table[i_tok])
        max_t = ['O', 'B', 'I'][viterbi_table[i_tok].index(max_v)]
    indices = {0: 'O', 1: 'B', 2: 'I'}
    # Read the best tags from the table and return the sequence
    for i in range(len(sequence)):
        max_labels.append(indices[viterbi_table[i].index(max(viterbi_table[i]))])
    return max_labels

The sequence tagger can now be trained on whichever corpus we like. To then find the most probably tags for a sequence, we can just use the viterbi algorithm,
which takes a list of tokens and our trained model as parameters and returns a sequence of
token-tag pairs:

In [5]:
def get_results(training: str):
    test = ['reman', 'gne', 'electoral_tweets']
    test.remove(training)
    hmm = HiddenMarkovModel()
    hmm.train_probabilities(train_set=data_source.loc[data_source['dataset']==f'{training}'])
    test_data = data_source.loc[(data_source['dataset'] == f'{test[0]}')|(data_source['dataset'] == f'{test[1]}')]

    out = [['predicted', 'gold']]
    ids = []
    for i, row in test_data.iterrows():
        sequence = list(row['tokens'])
        pred = viterbi(sequence, hmm)
        ids.append(row['id'])
        out.append([pred, list(row['target'])])

    out_df = pandas.DataFrame(out[1:], columns=out[0], index=ids)
    out_df.to_json(f'{training}_trained.json', orient='index')

### Trained on GoodNewsEveryone

In [6]:
get_results('gne')

### Trained on Reman

In [7]:
get_results('reman')

### Trained on Electoral tweets

In [8]:
get_results('electoral_tweets')